In [261]:
from qiskit import *
from qiskit.quantum_info import *
from qiskit.circuit.library import *

import numpy as np
from random import randrange

In [262]:
def beta_j_s(j, s, alpha):
    num = np.sqrt(sum([alpha[(((2*j)-1)*(2**(s-1)))+l]**1 for l in range(1,2**(s-1)+1)]))
    den = np.sqrt(sum([alpha[((j-1)*(2**s))+l]**1 for l in range(1,2**s+1)]))
    return (2*np.arcsin(num/den) if num else 0)

def prep_state(amplitudes):
    alpha = [0]+ amplitudes
    alpha = np.round(alpha/np.linalg.norm(alpha,1.0), 20)
    num_qubits = int(np.ceil(np.log2(len(alpha)-1)))
    total_len = 2**num_qubits
    alpha = np.append(alpha, [0]*(total_len-len(alpha)+1))

    h = {}

    s = 1
    for i in range(num_qubits):
        total = 2**(num_qubits-s)
        jlist = range(1, total+1)

        for j in jlist:
            h[(s,j)] = beta_j_s(j,s,alpha)

        s += 1

    QC = QuantumCircuit(num_qubits)
    for wire in range(num_qubits, 0, -1):
        for j in range(1, 2**(num_qubits-wire)+1):
            bitstring = format(j-1, "#010b")[-(num_qubits-wire):] if wire != num_qubits else ""
            print ("Beta (wire, j)", (wire,j),num_qubits-wire, "Rotation angle : ", h[(wire,j)])
            control_wires = []
            
            if not h[(wire,j)]: 
                continue

            for i,b in zip(bitstring, range(wire, num_qubits)):
                if i == '0': 
                    QC.x(b)
                control_wires.append(b)
                    
            G = RYGate(h[(wire,j)], label="RY(" + str(wire) + str(j) + ")").control(len(control_wires)) if len(control_wires) else RYGate(h[(wire,j)])
            QC.append(G, control_wires + [wire-1])

            for i,b in zip(bitstring, range(wire, num_qubits)):
                if i == '0': 
                    QC.x(b)

    return QC

In [263]:
# prep state 1

psiA = prep_state([0.2, 0.8])
state = Statevector(psiA)
state.draw(output="Latex")

Beta (wire, j) (1, 1) 0 Rotation angle :  2.214297435588181


<IPython.core.display.Latex object>

In [264]:
# prep state 1
psiB = prep_state([0.3, 0.7])
state = Statevector(psiB)
state.draw(output="Latex")

Beta (wire, j) (1, 1) 0 Rotation angle :  1.9823131728623846


<IPython.core.display.Latex object>

In [265]:
anc = QuantumRegister(1, name="anc")
pA = QuantumRegister(1, name="psiA")
pB = QuantumRegister(1, name="psiB")
classical = ClassicalRegister(1)

QC = QuantumCircuit(anc,pA, pB, classical)
QC.append(psiA, [1])
QC.append(psiB, [2])
QC.barrier()
QC.h(0)
QC.cswap(0,1,2)
QC.h(0)
QC.measure(0,0)
QC.draw()


░ ┌───┐   ┌───┐┌─┐
 anc: ────────────────░─┤ H ├─■─┤ H ├┤M├
      ┌─────────────┐ ░ └───┘ │ └───┘└╥┘
psiA: ┤ circuit-537 ├─░───────X───────╫─
      ├─────────────┤ ░       │       ║ 
psiB: ┤ circuit-540 ├─░───────X───────╫─
      └─────────────┘ ░               ║ 
c4: 1/════════════════════════════════╩═
                                      0

In [266]:
from qiskit.primitives import Sampler
S = Sampler().run(QC, shots=1000)
print (S.result())

SamplerResult(quasi_dists=[{0: 0.993, 1: 0.007}], metadata=[{'shots': 1000}])


$ P(0) = \frac{1}{2}((\langle a|b \rangle)^2 + 1) $

In [267]:
Prob0 = S.result().quasi_dists[0][0]
inner_product = np.sqrt(2*Prob0-1)
print (Prob0, 2*Prob0-1, inner_product)

0.993 0.986 0.9929753269845127
